# 실습 [23-1]<br>
**실습명: Bidirectional LSTM을 이용한 개체명 인식**<br>
- CoNLL-2003 데이터 사용해서 실습

데이터 형식에 대한 인사이트
1. 데이터는 [단어][품사][청크][개체명] 형식 (rejects VBZ B-VP O)
2. 4가지 개체명 (사람: B-PER, 장소: B-LOC, 기관: B-ORG, 기타: B-MISC)에 대한 태그 제공

실습 순서
1. 입력받은 문장을 토큰 단위로 분리
2. 분리된 단어의 형태소 분석
3. 형태소로 분리된 단어의 개체명 분석

In [ ]:
#관련 라이브러리 불러오기
import tensorflow
import keras
print("tensorflow version :", tensorflow.__version__)
print("keras version :", keras.__version__)

tensorflow version : 2.5.0
kera version : 2.5.0


In [12]:
#데이터 다운로드
from google.colab import files
uploaded = files.upload()

Saving train.txt to train.txt


In [13]:
import re
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences #padding 연산
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [40]:
#CoNLL-2003 train 데이터의 전처리 과정
sentences = []
buff_sentences = []

with open('./train.txt', 'r') as f_r:
  data = f_r.readlines() #문장 전체 불러오기
  for sentence in data: #각 문장에 대해서 처리
    if len(sentence) == 0 or sentence.startswith('-DOCSTART-') or sentence[0] == "\n": #sentence가 없거나 시작이거나 그냥 다음줄이면
      if len(buff_sentences) > 0:
        sentences.append(buff_sentences)
        buff_sentences = []
      continue
    
    splits = sentence.split(' ') #토큰 단위로 분리 (띄어쓰기 기준)
    splits[-1] = re.sub(r'\n', '', splits[-1]) #줄바꿈 표시 \n 제거
    word = splits[0].lower() #단어들은 모두 소문자로 바꿔서 저장
    buff_sentences.append([word, splits[-1]]) #buff_sentences = 단어, 개체명 태깅 정보 포함 (단어, 품사, 청크, 개체명 순이므로 -1로 개체명 불러오기)

In [41]:
#sentence = 현재 단어의 단어, 태깅 정보 포함하고 있음
#저장된 sentences 데이터 중 하나 예시로 뽑아보기
print(sentences[0])

#['단어', '개체명']의 형태로 갖고 있음 확인 가능

[['eu', 'B-ORG'], ['rejects', 'O'], ['german', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['british', 'B-MISC'], ['lamb', 'O'], ['.', 'O']]


In [42]:
#training 데이터를 만들기 위해 단어 - 개체명 분리해야 함
sentences_info = [] #단어 변수
tags_info = [] #품사 태깅 정보

for sent in sentences:
  word, tag = zip(*sent) #zip으로 단어와 품사 분리

  sentences_info.append(list(word))
  tags_info.append(list(tag))

In [43]:
print("첫 번째 문장 :", sentences_info[0])
print("첫 번째 품사 태깅 정보 :", tags_info[0])

첫 번째 문장 : ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
첫 번째 품사 태깅 정보 : ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [44]:
#인덱스 번호로 10번째 정보 추출해보기
print("11 번째 문장 :", sentences_info[10])
print("11 번째 품사 태깅 정보 :", tags_info[10])

11 번째 문장 : ['spanish', 'farm', 'minister', 'loyola', 'de', 'palacio', 'had', 'earlier', 'accused', 'fischler', 'at', 'an', 'eu', 'farm', 'ministers', "'", 'meeting', 'of', 'causing', 'unjustified', 'alarm', 'through', '"', 'dangerous', 'generalisation', '.', '"']
11 번째 품사 태깅 정보 : ['B-MISC', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [45]:
  #1) Tokenizer을 이용해서 토큰화/정수 인코딩
#vocabulary 생성을 위해 빈도수 가장 높은 5000개 단어만 추출
vocab_size = 5000

src_tokenizer = Tokenizer(num_words=vocab_size, oov_token='OOV') #문장 데이터
#OOV: Out of Vocabulary
src_tokenizer.fit_on_texts(sentences_info)

tag_tokenizer = Tokenizer() #태그 데이터
tag_tokenizer.fit_on_texts(tags_info)

In [46]:
#각 단어 id값으로 변환 -> 정수 인코딩
x_train = src_tokenizer.texts_to_sequences(sentences_info)
y_train = tag_tokenizer.texts_to_sequences(tags_info)

In [47]:
print(x_train[0]) #정수 인코딩 되어 있음 확인 가능
print(y_train[0])

[989, 1, 205, 629, 7, 3939, 216, 1, 3]
[4, 1, 7, 1, 1, 1, 7, 1, 1]


In [48]:
#vocabulary에 해당 단어가 없는 경우에는 OOV 처리함
index2word = src_tokenizer.index_word
index2tag = tag_tokenizer.index_word

decoded = []

for index in x_train[0]: #각 인덱스별로 단어 추출
  decoded.append(index2word[index])

print("기존 문장 : {}".format(sentences_info[0]))
print("Vocabulary에 없어 OOV 처리된 단어: {}".format(decoded))

#rejects, lamb 단어가 Vocabulary에 없어 OOV 처리됨을 알 수 있음

기존 문장 : ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
Vocabulary에 없어 OOV 처리된 단어: ['eu', 'OOV', 'german', 'call', 'to', 'boycott', 'british', 'OOV', '.']


In [49]:
  #2) Padding 연산으로 모든 문장 길이 동등하게 맞춰주기
#1번째와 11번째 문장만 봐도 서로 다른 길이임을 알 수 있음 -> 같은 길이로 맞추기 위해 패딩 연산
#padding: 단어/태그가 존재하지 않는 공간을 전부 0으로 채워 모든 문장의 길이를 동등하게 맞춰줌 (문장 길이=80)
max_len = 80

x_train_padded = pad_sequences(x_train, padding='post', maxlen=max_len)
y_train_padded = pad_sequences(y_train, padding='post', maxlen=max_len)

print(x_train_padded[0])
print(y_train_padded[0])

[ 989    1  205  629    7 3939  216    1    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]
[4 1 7 1 1 1 7 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]


In [50]:
#training 데이터 : test 데이터 8:2로 분할
x_train, x_test, y_train, y_test = train_test_split(x_train_padded, y_train_padded, test_size=0.2, random_state=555)

print(len(x_train))
print(len(y_train))

11232
11232


In [ ]:
#레이블 데이터(y)에 해당되는 태깅 정보에 one-hot encoding
tag_size = len(tag_tokenizer.word_index) + 1

y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

print(y_train[0])
print(y_test[0])

In [52]:
#각 데이터의 크기 확인
print('훈련 샘플 문장 크기: {}'.format(x_train.shape))
print('훈련 샘플 레이블 크기: {}'.format(y_train.shape))

print('테스트 샘플 문장 크기: {}'.format(x_test.shape))
print('테스트 샘플 레이블 크기: {}'.format(y_test.shape))

훈련 샘플 문장 크기: (11232, 80)
훈련 샘플 레이블 크기: (11232, 80, 10)
테스트 샘플 문장 크기: (2809, 80)
테스트 샘플 레이블 크기: (2809, 80, 10)


In [53]:
#Bi-LSTM을 이용한 개체명 인식
#Bi-LSTM 모델 생성 과정
from keras.models import Sequential #layer을 선형 형태로 구성하기 위한 도구
from keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from keras.optimizers import Adam

In [54]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len, mask_zero=True))
  #mask_zero=True : 패딩 연산에서 추가한 0 계산에서 제외
model.add(Bidirectional(LSTM(256, return_sequences=True))) #LSTM 양방향 학습
model.add(TimeDistributed(Dense(tag_size, activation='softmax'))) #softmax로 0~1 확률값으로 표현

In [55]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

신경망 학습시에 tensorflow 버전 2.0.0과 autograph 버전의 충돌로 인해 

```
WARNING:tensorflow:AutoGraph could not transform <function Model.make_train_function.<locals>.train_function at 0x7fe28cd89290> and will run it as-is.
```
이라고 뜬다. 이를 해결해주기 위해서는 gast를 설치해주면 된다:
- gast : generic Abstract Syntax Tree(AST)의 약자로, 다양한 파이썬 버전들 사이에 양립할 수 있는 층 (compatibility layer) 제공 라이브러리


In [62]:
!pip install gast==0.4.0

  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [65]:
import gast

In [66]:
#학습된 모델을 hist 변수에 넣어 history -> loss, accuracy 추출
hist = model.fit(x_train, y_train, batch_size=128, epochs=1, validation_data=(x_test, y_test))

88/88 [==============================] - 144s 2s/step - loss: 0.0853 - accuracy: 0.8576 - val_loss: 0.0659 - val_accuracy: 0.8861


In [67]:
print("train loss : ", hist.history['loss'])
print("train accuracy : ", hist.history['accuracy'])
print("val loss : ", hist.history['val_loss'])
print("val accuracy : ", hist.history['val_accuracy'])

train loss :  [0.08532189577817917]
train accuracy :  [0.8575764298439026]
val loss :  [0.06591226160526276]
val accuracy :  [0.8861467838287354]


In [ ]:
#테스트 정확도 출력
print("\n 테스트 정확도 : &0.4f " % (model.evaluate(x_test, y_test)[1]))